### Importing necessary libraries

In [1]:
from dotenv import load_dotenv
import os
from langchain_core.prompts.prompt import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
import json
from pathlib import Path
from operator import itemgetter

e:\Learnings\Comprehension-generator\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading Google API Key and Langchain API Key from environment variables

In [2]:
# Load environment variables from .env file
load_dotenv()

# Set the GOOGLE_API_KEY environment variable to the value retrieved from .env file
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

## Langsmith tracking
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

Initializing Large Language Model-Gemini-1.5-pro-latest

In [3]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest", temperature=0.1)

Loading JSON response file

In [4]:
path_absolute = Path("E:\Learnings\Comprehension-generator")
path_relative = Path("response.json")
full_path = path_absolute / path_relative

with open(full_path,'r') as f:
   RESPONSE_JSON = json.load(f)

In [5]:
RESPONSE_JSON

{'comprehension_passage': 'This is a sample comprehension passage. It contains several sentences providing information or a story.',
 'mcqs': [{'question': 'What is the capital of France?',
   'options': ['London', 'Paris', 'Berlin', 'Rome'],
   'answer_key': ''},
  {'question': "Who wrote 'To Kill a Mockingbird'?",
   'options': ['Harper Lee',
    'J.K. Rowling',
    'Stephen King',
    'Charles Dickens'],
   'answer_key': 0}],
 'short_answer_questions': [{'question': 'What is the main theme of the passage?',
   'demo_answer': 'The main theme of the passage is about...'},
  {'question': 'Explain the significance of the character X in the passage.',
   'demo_answer': 'The character X is significant because...'}],
 'long_answer_questions': [{'question': 'Discuss the impact of climate change on global agriculture.',
   'demo_answer': 'Climate change has a profound impact on global agriculture...'},
  {'question': 'Analyze the character development in the provided passage.',
   'demo_answ

Comprehension and question generation instruction

In [6]:
generate_instruction= """
You are an expert comprehension generator\
    Given a specific {topic} your task is to\  
    generate a comprehension passage within strictly following the given word range of {words} words \
    keeping in mind the difficulty level {difficulty}
    Next you have to have to generate MCQS, short answer type and long answer type questions.
    The number of questions for MCQ will be {mcq_no},do not generate if 0\
    for short answer type{sa_no}, do not generate if 0 and\
    for long answer type{la_no},do not generate if 0.
    Provide the demo answers for all the generated questions of MCQ, short answer and long answer questions at the end.
    The long type question demo answers should be of minimimum 100 words\
    Provide the output in the given format\
    There should not be any marker just the passage and the questions\
    {RESPONSE_JSON}
    """

Comprehension and question generation prompt template

In [7]:
generation_prompt= PromptTemplate(
    input_variables=['topic','words','difficulty','mcq_no','sa_no','la_no','RESPONSE_JSON'],
    template= generate_instruction
)

In [8]:
generation_prompt

PromptTemplate(input_variables=['RESPONSE_JSON', 'difficulty', 'la_no', 'mcq_no', 'sa_no', 'topic', 'words'], template='\nYou are an expert comprehension generator\\ Given a specific {topic} your task is to\\  \ngenerate a comprehension passage within {words} words keeping in mind the difficulty level {difficulty}\nNext you have to have to generate MCQS, short answer type and long answer type questions.\nThe number of questions for MCQ will be {mcq_no},for short answer type{sa_no} andfor long answer type{la_no}.\nProvide the demo answers for all the generated questions of MCQ, short answer and long answer questions at the end.\nThe long type question demo answers should be of minimimum 100 wordsProvide the output in the given format{RESPONSE_JSON}\nThe #RESPONSE_JSON keys and values should be enclosed within # double quotes""\n')

 Creating an instance of StrOutputParser for parsing the output

In [9]:
output_parser = StrOutputParser()

Generation chain

In [10]:
chain_generation=generation_prompt| llm| output_parser 

In [11]:
chain_generation

PromptTemplate(input_variables=['RESPONSE_JSON', 'difficulty', 'la_no', 'mcq_no', 'sa_no', 'topic', 'words'], template='\nYou are an expert comprehension generator\\ Given a specific {topic} your task is to\\  \ngenerate a comprehension passage within {words} words keeping in mind the difficulty level {difficulty}\nNext you have to have to generate MCQS, short answer type and long answer type questions.\nThe number of questions for MCQ will be {mcq_no},for short answer type{sa_no} andfor long answer type{la_no}.\nProvide the demo answers for all the generated questions of MCQ, short answer and long answer questions at the end.\nThe long type question demo answers should be of minimimum 100 wordsProvide the output in the given format{RESPONSE_JSON}\nThe #RESPONSE_JSON keys and values should be enclosed within # double quotes""\n')
| ChatGoogleGenerativeAI(model='gemini-pro', client=genai.GenerativeModel(
      model_name='models/gemini-pro',
      generation_config={},
      safety_sett

Review instructions

In [12]:
review_instruction= """
    As an expert in English grammar and writing, your task is to assess a comprehension passage{response}\
    The passage provided is {words} words long and pertains to the topic of {topic}\
    Your evaluation should include an assessment of the passage's difficulty level, as specified {difficulty}\
    Thoroughly check the number of words in the passage as mentioned {words} words  is followed correctly\
    Furthermore, you are required to evaluate multiple-choice questions, short-answer questions, and long-answer questions embedded within the passage\
    If any of the questions do not align with the cognitive and analytical abilities expected of the students, you must suggest changes to ensure they are suitable\
    Additionally, you are to evaluate the provided answers to the questions, ensuring they are accurate and appropriate.
    Please ensure that the output consists solely of the updated comprehension passage and the questions along with their answers\
    as per the given format {RESPONSE_JSON}\
    Ensure that in the {response},commprehension passage and the demo answers does not contain unnecessary line gaps  \
    Also  make sure that the dictionary is enclosed with quotes\
    """

Review prompt template

In [13]:
review_template= PromptTemplate(
    input_variables=['topic','words','difficulty','response','RESPONSE_JSON'],
    template= review_instruction
)

In [14]:
review_template

PromptTemplate(input_variables=['RESPONSE_JSON', 'difficulty', 'response', 'topic', 'words'], template="\nAs an expert in English grammar and writing, your task is to assess a comprehension passage{response}The passage provided is {words} words long and pertains to the topic of {topic}Your evaluation should include an assessment of the passage's difficulty level, as specified {difficulty}Furthermore, you are required to evaluate multiple-choice questions, short-answer questions, and long-answer questions embedded within the passageIf any of the questions do not align with the cognitive and analytical abilities expected of the students, you must suggest changes to ensure they are suitableAdditionally, you are to evaluate the provided answers to the questions, ensuring they are accurate and appropriate.\nPlease ensure that the output consists solely of the updated comprehension passage and the questions along with their answersas per the given format {RESPONSE_JSON}In the output format k

Combining generation chain with review chain

In [15]:
review_chain= (
    {"response": chain_generation, "difficulty": itemgetter("difficulty"), "topic": itemgetter("topic"), "words": itemgetter("words"), "RESPONSE_JSON": itemgetter("RESPONSE_JSON")}
    | review_template
    | llm
    | StrOutputParser() 
)

In [16]:
review_chain

{
  response: PromptTemplate(input_variables=['RESPONSE_JSON', 'difficulty', 'la_no', 'mcq_no', 'sa_no', 'topic', 'words'], template='\nYou are an expert comprehension generator\\ Given a specific {topic} your task is to\\  \ngenerate a comprehension passage within {words} words keeping in mind the difficulty level {difficulty}\nNext you have to have to generate MCQS, short answer type and long answer type questions.\nThe number of questions for MCQ will be {mcq_no},for short answer type{sa_no} andfor long answer type{la_no}.\nProvide the demo answers for all the generated questions of MCQ, short answer and long answer questions at the end.\nThe long type question demo answers should be of minimimum 100 wordsProvide the output in the given format{RESPONSE_JSON}\nThe #RESPONSE_JSON keys and values should be enclosed within # double quotes""\n')
            | ChatGoogleGenerativeAI(model='gemini-pro', client=genai.GenerativeModel(
                  model_name='models/gemini-pro',
       

Invoking the review chain to generate a refined output

In [17]:
response= review_chain.invoke({'RESPONSE_JSON':RESPONSE_JSON, 'difficulty':'easy', 'la_no':2, 'mcq_no':3, 'sa_no':1, 'topic':'India', 'words':300 })

In [18]:
print(response)

{
 "comprehension_passage": "India is a country in South Asia. It is the second most populous country in the world, with a population of over 1.3 billion people. India is a diverse country, with many different languages, religions, and cultures. The official languages of India are Hindi and English. The capital of India is New Delhi. India is a democracy, with a parliamentary system of government. The President of India is the head of state, and the Prime Minister of India is the head of government.",
 "mcqs": [
  {
   "question": "What is the capital of India?",
   "options": [
    "Mumbai",
    "New Delhi",
    "Kolkata",
    "Chennai"
   ],
   "answer_key": "New Delhi"
  },
  {
   "question": "What is the official language of India?",
   "options": [
    "Hindi",
    "English",
    "Tamil",
    "Telugu"
   ],
   "answer_key": "Hindi"
  },
  {
   "question": "What type of government does India have?",
   "options": [
    "Monarchy",
    "Democracy",
    "Dictatorship",
    "Oligarchy

Converting the review chain response to JSON

In [19]:
response= json.loads(response)

In [20]:
response

{'comprehension_passage': 'India is a country in South Asia. It is the second most populous country in the world, with a population of over 1.3 billion people. India is a diverse country, with many different languages, religions, and cultures. The official languages of India are Hindi and English. The capital of India is New Delhi. India is a democracy, with a parliamentary system of government. The President of India is the head of state, and the Prime Minister of India is the head of government.',
 'mcqs': [{'question': 'What is the capital of India?',
   'options': ['Mumbai', 'New Delhi', 'Kolkata', 'Chennai'],
   'answer_key': 'New Delhi'},
  {'question': 'What is the official language of India?',
   'options': ['Hindi', 'English', 'Tamil', 'Telugu'],
   'answer_key': 'Hindi'},
  {'question': 'What type of government does India have?',
   'options': ['Monarchy', 'Democracy', 'Dictatorship', 'Oligarchy'],
   'answer_key': 'Democracy'}],
 'short_answer_questions': [{'question': 'What